In [8]:
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing.pool import Pool

from collections import OrderedDict
import sys
import os
print(os.getcwd())
sys.path.insert(0, os.getcwd()+'/...') # +'/..'
sys.path.insert(0, os.getcwd()+'/..') # +'/..'
sys.path.insert(0, os.getcwd()+'/') # +'/..'
from IPython.display import clear_output

/home/sheikh/Motius/Thesis/synch_signal_detection_dl/tools/sionna/examples/myTests


In [12]:
import tensorflow as tf
from sionna.fec.scrambling import Scrambler
from sionna.fec.scrambling import Descrambler


In [239]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer

class IEEE80211Scrambler(Layer):
    
    def __init__(self, seed = 127, binary = True, dtype  = tf.int32,**kwargs ):
        
        self._binary  = binary
        if seed is not None:
            self.seed  = seed
        else:
            self.seed   = 127
        
        self.sequence = None
        
        super().__init__(dtype=dtype, **kwargs)

    def _de2bi(self, n, N):
        bseed= bin(n).replace("0b", "")
        fix = N-len(bseed)
        pad = np.zeros(fix)
        pad = pad.tolist()
        y = []
        for i in range(len(pad)):
            y = [int(pad[i])] + y
        for i in range(len(bseed)):
            y = [int(bseed[i])] + y
        return y
            
    def _generate_scrambling(self, len_input_shape):
        
        bit_count =len_input_shape # len(bits)
        
        N=7
        bseed = self._de2bi(self.seed, N)
        bseed = np.array(bseed, dtype=int)
        x = np.zeros((N,), dtype = int)
        x = bseed
        var_arr = np.zeros(bit_count, dtype = int)
        
        
        for n in range(bit_count):
            var = int(x[3])^int(x[6])
            x[1:] = x[0:6]
            x[0] = var
            var_arr[n] = var
        
        self.sequence = tf.convert_to_tensor(var_arr)
        
        

                
            
    def build(self, input_shape):
        """Build the model and initialize variables."""
        pass
    
    def call(self, inputs):
        is_binary = self._binary # can be overwritten if explicitly provided

        if isinstance(inputs, (tuple, list)):
            if len(inputs)==1: # if user wants to call with call([x])
                seed = None
                x = inputs
            elif len(inputs)==2:
                x, seed = inputs
            elif len(inputs)==3:
            # allow that flag binary is explicitly provided (for descrambler)
                x, seed, is_binary = inputs
                # is binary can be either a tensor or bool
                if isinstance(is_binary, tf.Tensor):
                    if not is_binary.dtype.is_bool:
                        raise TypeError("binary must be bool.")
                else: # is boolean
                    assert isinstance(is_binary.dtype, bool), \
                    "binary must be bool."
            else:
                raise TypeError("inputs cannot have more than 3 entries.")
        else:
            seed = self.seed
            x = inputs
        
        self.len_input_shape = len(x)
        self.seed = seed
        self._generate_scrambling( self.len_input_shape)
        
        
        scrambled_bits = tf.zeros(x.shape[0], dtype = tf.int32)
        scrambled_bits = tf.bitwise.bitwise_xor(self.sequence,x )
        return scrambled_bits





# def  scrambler(bits,seed):
#     """
#     Using an initial state of all 1s: [1 1 1 1 1 1] and bits = np.zeros(127,)
#     The scrambler should generate a 127-bit sequence as follows:
#      out = [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
#              0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
#              0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
#              1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
#              1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
#              1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
#              0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
#              1]
#     And descrambler will work in reverse with the same sequence.
#     Args:
#         bits: input bits (int)
#         seed: A pseudo-random seed of 7 bits, default = [1 0 1 1 1 0 1]
#     Reference: https://edadocs.software.keysight.com/display/ads2009/WLAN+Scrambler
#     https://uk.mathworks.com/help/wlan/ref/wlanwaveformgenerator.html#:~:text=Scrambler%20Initialization-,The%20scrambler%20initialization,-used%20on%20the

#     Returns:

#     """
#     #predict initial state
#     # initialize scrambler
#     # scramble
    
#     bit_count = len(bits)
#     scrambled_bits = np.zeros(bit_count, dtype = int)
#     N=7
#     bseed = de2bi(seed,N)
#     bseed = np.array(bseed, dtype=int)
#     x = np.zeros((N,), dtype = int)
#     x = bseed
#     var_arr = np.zeros(bit_count, dtype = int)
    
    
#     for n in range(bit_count):
#         var = int(x[3])^int(x[6])
#         x[1:] = x[0:6]
#         x[0] = var
#         var_arr[n] = var
        
#     scrambled_bits =np.bitwise_xor(var_arr,bits)

#     return scrambled_bits, var_arr



# VERIFY = True
# out_valid = [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
#         0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
#         0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
#         1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
#         1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
#         1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
#         0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
#         1]
# if VERIFY:
#     bits = np.zeros((127,), dtype = int)
#     seed = [1, 1, 1, 1, 1, 1, 1]
#     seed = bi2de(seed)
#     out, var_arr = scrambler(bits,seed)
#     if np.array_equal(out_valid,out):
#         print("valid scrambler!")
#     bits_out,var_arr  =  scrambler(out,seed)   
#     if np.array_equal(bits,bits_out):
#         print("valid de-scrambler!")



In [257]:
scrambler = IEEE80211Scrambler(seed = 127)
bits = np.zeros((127*3,), dtype = int)
seed  = 127
out = scrambler([bits,50])
out


<tf.Tensor: shape=(381,), dtype=int64, numpy=
array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
     

In [259]:
out2 = scrambler([out, 50])
out2

<tf.Tensor: shape=(381,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     